In [1]:
import sqlite3
import os.path
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', None)

#SQLlite does not have the Power() function
#So I am implementing it from scratch
def sqlite_power(x,n):
    return int(x)**n

In [2]:
if os.path.isfile('Revolut.db'):
    conn = sqlite3.connect('Revolut.db') 
    conn.create_function("power", 2, sqlite_power) 
    c = conn.cursor()

else:
    conn = sqlite3.connect('Revolut.db') 
    conn.create_function("power", 2, sqlite_power) 
    c = conn.cursor()

    read_countries = pd.read_csv ('countries.csv')
    read_countries.to_sql('COUNTRIES', conn)  

    read_currencydetails = pd.read_csv('currency_details.csv')
    read_currencydetails.to_sql('CURRENCY_DETAILS', conn)

    read_fraudsters = pd.read_csv('fraudsters.csv')
    read_fraudsters.to_sql('FRAUDSTERS', conn)

    read_fx_rates = pd.read_csv('fx_rates.csv')
    read_fx_rates.to_sql('FX_RATES', conn)

    read_transactions = pd.read_csv('transactions.csv')
    read_transactions.to_sql('TRANSACTIONS', conn)

    read_users = pd.read_csv('users.csv')
    read_users.to_sql('USERS', conn)

## 1) Examine query and repair it.

In [3]:
c.execute('''

WITH PROCESSED_USERS 
    AS (SELECT substr(u.phone_country, 1, 2) AS short_phone_country, u.id FROM USERS u)

SELECT t.user_id, t.merchant_country, SUM(t.amount/fx.rate/Power(10, cd.exponent)) AS amount

FROM TRANSACTIONS t
    JOIN FX_RATES fx
        ON (fx.base_ccy = t.currency AND fx.ccy = 'EUR')
    JOIN CURRENCY_DETAILS cd
        ON cd.currency = t.currency
    JOIN PROCESSED_USERS pu
        ON pu.id = t.user_id
        
WHERE t.source = 'GAIA' 

GROUP BY t.user_id, t.merchant_country

ORDER BY amount DESC
''')


df = DataFrame(c.fetchall())

df.head()

,0,1,2
0,dc283b17-bbe1-4ae9-a11c-0029d5ae71d9,USA,511649.977932
1,dc283b17-bbe1-4ae9-a11c-0029d5ae71d9,GBR,137274.496805
2,605452f6-e9ce-42ef-bbf4-71e47958dcee,SWE,111289.085234
3,4366241f-0cac-4978-a4a3-27f38abc174a,GBR,91353.437369
4,64bb9cd9-a55e-4e3c-86fb-13b74bebc603,PHL,64553.871168


## 2) Write a query to identify users whose first transaction was a successful card payment over $10 USD equivalent

In [4]:
c.execute('''

SELECT t.user_id,
       t.amount, t.currency,
       t.amount/fx.rate/Power(10, cd.exponent),
       MIN(t.created_date), t.source, t.state, t.merchant_category
       
       
FROM TRANSACTIONS t
    JOIN FX_RATES fx
        ON (fx.base_ccy = t.currency AND fx.ccy = 'USD')
    JOIN CURRENCY_DETAILS cd
        ON cd.currency = t.currency
    JOIN USERS u
        ON u.id = t.user_id
        
WHERE t.state = 'COMPLETED'
    AND t.source = 'GAIA'
    AND t.amount/fx.rate/Power(10, cd.exponent) >10

GROUP BY t.user_id
''')


df = DataFrame(c.fetchall())

df.head()

,0,1,2,3,4,5,6,7
0,001032e0-8071-4baf-95b9-e50214665c2e,995,EUR,11.558516,2017-09-04 16:19:05.416000,GAIA,COMPLETED,point_of_interest
1,00131af8-66f0-4526-8b5f-dc2fdb26c7d7,3700,EUR,42.981417,2018-07-06 12:16:44.927000,GAIA,COMPLETED,cafe
2,001926be-3245-43fa-86dd-b40ee160b6f9,3472,GBP,45.183647,2017-04-01 10:39:15.688000,GAIA,COMPLETED,clothing_store
3,0022f893-47c7-4da0-96df-7ea564bfd50d,943,GBP,12.271941,2018-01-26 20:07:12.266000,GAIA,COMPLETED,supermarket
4,002ad534-53c5-4320-a199-45a2b0a9265a,18695,GBP,243.291554,2018-02-24 13:32:30.740000,GAIA,COMPLETED,None


## 3) Find fraudsters

In [5]:
c.execute('''

SELECT t.user_id, COUNT(*) AS count
       
FROM TRANSACTIONS t 
    JOIN FRAUDSTERS f 
        ON f.user_id = t.user_id
    
WHERE t.state = 'DECLINED' OR t.state = 'REVERTED' OR t.state = 'FAILED'

GROUP BY t.user_id

ORDER BY count DESC
''')

fraudsters = DataFrame(c.fetchall())

fraudsters.head()

,0,1
0,dc283b17-bbe1-4ae9-a11c-0029d5ae71d9,417
1,eab966ce-5539-4a89-8be5-67b1e359203d,188
2,6eb51e3f-6e8b-45ee-bf01-82495df9c930,176
3,c9f032a6-8383-4a96-aeaa-18aa3f57dcde,107
4,c51c8f21-2361-4550-beca-ae0dfb411e59,85


In [6]:
fraudsters.shape

(288, 2)

In [7]:
c.execute('''

SELECT t.user_id, COUNT(*) AS count
       
FROM TRANSACTIONS t
    
WHERE t.state = 'DECLINED' 
      OR t.state = 'REVERTED' 
      OR t.state = 'FAILED'

GROUP BY t.user_id

ORDER BY count DESC

LIMIT 30
''')

likely_fraudsters = DataFrame(c.fetchall())

In [8]:
likely_fraudsters

,0,1
0,dc283b17-bbe1-4ae9-a11c-0029d5ae71d9,417
1,46172727-471c-4627-b706-1f9881a8e4d2,370
2,06bb2d68-bf61-4030-8447-9de64d3ce490,367
3,c9d85060-24a7-4783-a911-582e186bd4eb,344
4,65ac0928-e17d-4636-96f4-ebe6bdb9c98d,317
5,5fd70cca-1685-40ad-a312-7d387f6911a9,305
6,8ca1b9ce-8d58-416f-90eb-6c666fc866bd,283
7,f54baeeb-7282-4d23-9bb7-e8396ce1b159,279
8,62aaa0fb-65ae-44b6-884d-56f38a302b3e,274
9,605452f6-e9ce-42ef-bbf4-71e47958dcee,272


In [9]:
likely_fraudsters['Is in fraudsters.csv'] = likely_fraudsters[0].isin(fraudsters[0])

In [10]:
likely_fraudsters

,0,1,Is in fraudsters.csv
0,dc283b17-bbe1-4ae9-a11c-0029d5ae71d9,417,True
1,46172727-471c-4627-b706-1f9881a8e4d2,370,False
2,06bb2d68-bf61-4030-8447-9de64d3ce490,367,False
3,c9d85060-24a7-4783-a911-582e186bd4eb,344,False
4,65ac0928-e17d-4636-96f4-ebe6bdb9c98d,317,False
5,5fd70cca-1685-40ad-a312-7d387f6911a9,305,False
6,8ca1b9ce-8d58-416f-90eb-6c666fc866bd,283,False
7,f54baeeb-7282-4d23-9bb7-e8396ce1b159,279,False
8,62aaa0fb-65ae-44b6-884d-56f38a302b3e,274,False
9,605452f6-e9ce-42ef-bbf4-71e47958dcee,272,False
